# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.13.0 opencv-python mediapipe scikit-learn matplotlib pygame

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Guru1', 'Guru2', 'Guru3', 'Guru4', 'Guru5', 'Guru6', 'Guru7', 'Guru8', 'Stand'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [9]:
for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    #for sequence in range(1,no_sequences+1):
     for sequence in range(no_sequences):
        try: 
            #os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [10]:
pygame.init()
pygame.mixer.init()
ready_sound=pygame.mixer.Sound("getready.wav")
start_sound=pygame.mixer.Sound("startpose.wav")
end_sound=pygame.mixer.Sound("endpose.wav")

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.9) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            #ready_sound.play()
            #cv2.waitKey(3000)
            #start_sound.play()
            #cv2.waitKey(100)
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)   
                    cv2.putText(image, 'STARTING', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            #end_sound.play()
            #cv2.waitKey(3000)
        cv2.waitKey(30000)
        #temp = input("Go to the next pose")
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1711303933.488045       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [11]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'Guru1': 0,
 'Guru2': 1,
 'Guru3': 2,
 'Guru4': 3,
 'Guru5': 4,
 'Guru6': 5,
 'Guru7': 6,
 'Guru8': 7,
 'Stand': 8}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
# Ok to skip
np.array(sequences).shape

(270, 30, 1662)

In [14]:
# Ok to skip
np.array(labels).shape

(270,)

In [15]:
# Ok to skip
X = np.array(sequences)

In [16]:
# Ok to skip
X.shape

(270, 30, 1662)

In [17]:
# Ok to skip
y = to_categorical(labels).astype(int)

In [18]:
# Ok to skip
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [19]:
# Ok to skip
y_test.shape

(14, 9)

# 7. Build and Train LSTM Neural Network

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [21]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [22]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=878, callbacks=[tb_callback])

Epoch 1/878
8/8 [==============================] - 5s 56ms/step - loss: 10.1988 - categorical_accuracy: 0.1484
Epoch 2/878
8/8 [==============================] - 0s 52ms/step - loss: 10.3836 - categorical_accuracy: 0.0977
Epoch 3/878
8/8 [==============================] - 0s 52ms/step - loss: 5.2788 - categorical_accuracy: 0.1016
Epoch 4/878
8/8 [==============================] - 0s 54ms/step - loss: 5.1624 - categorical_accuracy: 0.1133
Epoch 5/878
8/8 [==============================] - 0s 52ms/step - loss: 2.4900 - categorical_accuracy: 0.1328
Epoch 6/878
8/8 [==============================] - 0s 49ms/step - loss: 2.2311 - categorical_accuracy: 0.1719
Epoch 7/878
8/8 [==============================] - 0s 49ms/step - loss: 2.1883 - categorical_accuracy: 0.2109
Epoch 8/878
8/8 [==============================] - 0s 49ms/step - loss: 2.1621 - categorical_accuracy: 0.1875
Epoch 9/878
8/8 [==============================] - 0s 50ms/step - loss: 2.1383 - categorical_accuracy: 0.1836
Epoch 10

8/8 [==============================] - 0s 51ms/step - loss: 0.2382 - categorical_accuracy: 0.9062
Epoch 75/878
8/8 [==============================] - 0s 52ms/step - loss: 0.2203 - categorical_accuracy: 0.9258
Epoch 76/878
8/8 [==============================] - 0s 51ms/step - loss: 0.2733 - categorical_accuracy: 0.9219
Epoch 77/878
8/8 [==============================] - 0s 52ms/step - loss: 0.2118 - categorical_accuracy: 0.9297
Epoch 78/878
8/8 [==============================] - 0s 51ms/step - loss: 0.1814 - categorical_accuracy: 0.9453
Epoch 79/878
8/8 [==============================] - 0s 51ms/step - loss: 0.2521 - categorical_accuracy: 0.9102
Epoch 80/878
8/8 [==============================] - 0s 52ms/step - loss: 0.2582 - categorical_accuracy: 0.8906
Epoch 81/878
8/8 [==============================] - 0s 52ms/step - loss: 0.2750 - categorical_accuracy: 0.8828
Epoch 82/878
8/8 [==============================] - 0s 51ms/step - loss: 0.2814 - categorical_accuracy: 0.8945
Epoch 83/878
8

8/8 [==============================] - 0s 51ms/step - loss: 0.2054 - categorical_accuracy: 0.9531
Epoch 148/878
8/8 [==============================] - 0s 52ms/step - loss: 0.2071 - categorical_accuracy: 0.9258
Epoch 149/878
8/8 [==============================] - 0s 52ms/step - loss: 0.3266 - categorical_accuracy: 0.8672
Epoch 150/878
8/8 [==============================] - 0s 53ms/step - loss: 0.3993 - categorical_accuracy: 0.8672
Epoch 151/878
8/8 [==============================] - 0s 52ms/step - loss: 0.2606 - categorical_accuracy: 0.9062
Epoch 152/878
8/8 [==============================] - 0s 51ms/step - loss: 0.1879 - categorical_accuracy: 0.9570
Epoch 153/878
8/8 [==============================] - 0s 52ms/step - loss: 0.1789 - categorical_accuracy: 0.9570
Epoch 154/878
8/8 [==============================] - 0s 51ms/step - loss: 0.2005 - categorical_accuracy: 0.9414
Epoch 155/878
8/8 [==============================] - 0s 52ms/step - loss: 0.2467 - categorical_accuracy: 0.9102
Epoch 

8/8 [==============================] - 0s 51ms/step - loss: 0.1382 - categorical_accuracy: 0.9570
Epoch 221/878
8/8 [==============================] - 0s 52ms/step - loss: 0.1326 - categorical_accuracy: 0.9570
Epoch 222/878
8/8 [==============================] - 0s 52ms/step - loss: 0.3726 - categorical_accuracy: 0.8711
Epoch 223/878
8/8 [==============================] - 0s 51ms/step - loss: 0.5037 - categorical_accuracy: 0.8164
Epoch 224/878
8/8 [==============================] - 0s 52ms/step - loss: 0.4800 - categorical_accuracy: 0.8320
Epoch 225/878
8/8 [==============================] - 0s 53ms/step - loss: 0.2960 - categorical_accuracy: 0.8945
Epoch 226/878
8/8 [==============================] - 0s 53ms/step - loss: 0.1736 - categorical_accuracy: 0.9648
Epoch 227/878
8/8 [==============================] - 0s 52ms/step - loss: 0.1686 - categorical_accuracy: 0.9531
Epoch 228/878
8/8 [==============================] - 0s 53ms/step - loss: 0.1195 - categorical_accuracy: 0.9609
Epoch 

8/8 [==============================] - 0s 53ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 294/878
8/8 [==============================] - 0s 53ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 295/878
8/8 [==============================] - 0s 53ms/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 296/878
8/8 [==============================] - 0s 52ms/step - loss: 0.0016 - categorical_accuracy: 1.0000
Epoch 297/878
8/8 [==============================] - 0s 52ms/step - loss: 0.0015 - categorical_accuracy: 1.0000
Epoch 298/878
8/8 [==============================] - 0s 53ms/step - loss: 0.0016 - categorical_accuracy: 1.0000
Epoch 299/878
8/8 [==============================] - 0s 52ms/step - loss: 0.0020 - categorical_accuracy: 1.0000
Epoch 300/878
8/8 [==============================] - 0s 53ms/step - loss: 0.0015 - categorical_accuracy: 1.0000
Epoch 301/878
8/8 [==============================] - 0s 52ms/step - loss: 0.0014 - categorical_accuracy: 1.0000
Epoch 

...

8/8 [==============================] - 0s 53ms/step - loss: 1.5047e-05 - categorical_accuracy: 1.0000
Epoch 575/878
8/8 [==============================] - 0s 53ms/step - loss: 1.5014e-05 - categorical_accuracy: 1.0000
Epoch 576/878
8/8 [==============================] - 0s 53ms/step - loss: 1.4827e-05 - categorical_accuracy: 1.0000
Epoch 577/878
8/8 [==============================] - 0s 53ms/step - loss: 1.4692e-05 - categorical_accuracy: 1.0000
Epoch 578/878
8/8 [==============================] - 0s 53ms/step - loss: 1.4581e-05 - categorical_accuracy: 1.0000
Epoch 579/878
8/8 [==============================] - 0s 52ms/step - loss: 1.4524e-05 - categorical_accuracy: 1.0000
Epoch 580/878
8/8 [==============================] - 0s 53ms/step - loss: 1.4411e-05 - categorical_accuracy: 1.0000
Epoch 581/878
8/8 [==============================] - 0s 53ms/step - loss: 1.4404e-05 - categorical_accuracy: 1.0000
Epoch 582/878
8/8 [==============================] - 0s 52ms/step - loss: 1.4213e-05 -

8/8 [==============================] - 0s 54ms/step - loss: 9.6118e-06 - categorical_accuracy: 1.0000
Epoch 645/878
8/8 [==============================] - 0s 53ms/step - loss: 9.5014e-06 - categorical_accuracy: 1.0000
Epoch 646/878
8/8 [==============================] - 0s 54ms/step - loss: 9.6495e-06 - categorical_accuracy: 1.0000
Epoch 647/878
8/8 [==============================] - 0s 53ms/step - loss: 9.3994e-06 - categorical_accuracy: 1.0000
Epoch 648/878
8/8 [==============================] - 0s 53ms/step - loss: 9.4167e-06 - categorical_accuracy: 1.0000
Epoch 649/878
8/8 [==============================] - 0s 53ms/step - loss: 9.3892e-06 - categorical_accuracy: 1.0000
Epoch 650/878
8/8 [==============================] - 0s 53ms/step - loss: 9.2379e-06 - categorical_accuracy: 1.0000
Epoch 651/878
8/8 [==============================] - 0s 54ms/step - loss: 9.2062e-06 - categorical_accuracy: 1.0000
Epoch 652/878
8/8 [==============================] - 0s 53ms/step - loss: 9.1853e-06 -

8/8 [==============================] - 0s 53ms/step - loss: 6.4115e-06 - categorical_accuracy: 1.0000
Epoch 715/878
8/8 [==============================] - 0s 53ms/step - loss: 6.4003e-06 - categorical_accuracy: 1.0000
Epoch 716/878
8/8 [==============================] - 0s 53ms/step - loss: 6.3822e-06 - categorical_accuracy: 1.0000
Epoch 717/878
8/8 [==============================] - 0s 54ms/step - loss: 6.3263e-06 - categorical_accuracy: 1.0000
Epoch 718/878
8/8 [==============================] - 0s 53ms/step - loss: 6.2891e-06 - categorical_accuracy: 1.0000
Epoch 719/878
8/8 [==============================] - 0s 53ms/step - loss: 6.2453e-06 - categorical_accuracy: 1.0000
Epoch 720/878
8/8 [==============================] - 0s 53ms/step - loss: 6.2132e-06 - categorical_accuracy: 1.0000
Epoch 721/878
8/8 [==============================] - 0s 55ms/step - loss: 6.1820e-06 - categorical_accuracy: 1.0000
Epoch 722/878
8/8 [==============================] - 0s 53ms/step - loss: 6.1894e-06 -

8/8 [==============================] - 0s 53ms/step - loss: 4.4866e-06 - categorical_accuracy: 1.0000
Epoch 785/878
8/8 [==============================] - 0s 53ms/step - loss: 4.4000e-06 - categorical_accuracy: 1.0000
Epoch 786/878
8/8 [==============================] - 0s 52ms/step - loss: 4.3837e-06 - categorical_accuracy: 1.0000
Epoch 787/878
8/8 [==============================] - 0s 53ms/step - loss: 4.3823e-06 - categorical_accuracy: 1.0000
Epoch 788/878
8/8 [==============================] - 0s 53ms/step - loss: 4.3604e-06 - categorical_accuracy: 1.0000
Epoch 789/878
8/8 [==============================] - 0s 53ms/step - loss: 4.3115e-06 - categorical_accuracy: 1.0000
Epoch 790/878
8/8 [==============================] - 0s 53ms/step - loss: 4.2882e-06 - categorical_accuracy: 1.0000
Epoch 791/878
8/8 [==============================] - 0s 54ms/step - loss: 4.2700e-06 - categorical_accuracy: 1.0000
Epoch 792/878
8/8 [==============================] - 0s 53ms/step - loss: 4.2556e-06 -

8/8 [==============================] - 0s 54ms/step - loss: 3.0715e-06 - categorical_accuracy: 1.0000
Epoch 855/878
8/8 [==============================] - 0s 55ms/step - loss: 3.1101e-06 - categorical_accuracy: 1.0000
Epoch 856/878
8/8 [==============================] - 0s 54ms/step - loss: 3.0435e-06 - categorical_accuracy: 1.0000
Epoch 857/878
8/8 [==============================] - 0s 54ms/step - loss: 3.0701e-06 - categorical_accuracy: 1.0000
Epoch 858/878
8/8 [==============================] - 0s 54ms/step - loss: 3.0440e-06 - categorical_accuracy: 1.0000
Epoch 859/878
8/8 [==============================] - 0s 54ms/step - loss: 3.0207e-06 - categorical_accuracy: 1.0000
Epoch 860/878
8/8 [==============================] - 0s 55ms/step - loss: 2.9988e-06 - categorical_accuracy: 1.0000
Epoch 861/878
8/8 [==============================] - 0s 55ms/step - loss: 3.0258e-06 - categorical_accuracy: 1.0000
Epoch 862/878
8/8 [==============================] - 0s 55ms/step - loss: 2.9546e-06 -

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 9)                 297       
                                                                 
Total params: 596873 (2.28 MB)
Trainable params: 596873 

# 8. Make Predictions

In [26]:
res = model.predict(X_test)

1/1 [==============================] - 1s 541ms/step


In [27]:
actions[np.argmax(res[2])]

'Guru6'

In [28]:
actions[np.argmax(y_test[2])]

'Guru6'

# 9. Save Weights

In [29]:
model.save('action.h5')

C:\Users\svara\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
del model

In [31]:
# Once training is done, u do not have to train it again (no need to call the fit() finction). Just need to setup the model
# and compile it. then load the saved weights.
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [32]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [33]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [34]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [35]:
yhat = model.predict(X_test)

1/1 [==============================] - 1s 521ms/step


In [36]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [37]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[11,  0],
        [ 0,  3]],

       [[11,  0],
        [ 0,  3]],

       [[12,  0],
        [ 0,  2]],

       [[11,  1],
        [ 1,  1]],

       [[12,  0],
        [ 1,  1]],

       [[12,  1],
        [ 0,  1]],

       [[13,  0],
        [ 0,  1]]], dtype=int64)

In [38]:
accuracy_score(ytrue, yhat)

0.8571428571428571

# 11. Test in Real Time

In [39]:
from scipy import stats

In [40]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [41]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'image' is not defined

<Figure size 1800x1800 with 0 Axes>

In [45]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        #cv2.putText(image, ' '.join(sentence), (3,30), 
        #               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1711306134.775561       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
